# Data import and preprocessing for Experiment 2

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
data_folder= 'Experiment 2. Illusory Triangle and Common Region'

## Extracting finger-onset asynchrony: response time brackets for two- and three-target disappearance and appearance events

In [7]:
## change in physical state
State= pd.read_csv(os.path.join(data_folder, 'Experiment2_FOA_DisplayState.csv'), sep=';')
State['TargetCount']=State.Target1+State.Target2+State.Target3
State['Trial']= int(0)
State['ChangeMagnitude']= int(0)
for current_ID in State.ID.unique():
    for current_block in State[State.ID==current_ID].Block.unique():
        block_trials= State[(State.ID==current_ID) & (State.Block==current_block)].index
        for iTrial, iRow in enumerate(block_trials):
            State.set_value(iRow, 'Trial', iTrial)
            if iTrial>0:
                State.set_value(iRow, 'ChangeMagnitude', State.TargetCount[iRow]-State.TargetCount[iRow-1])     
                
# splitting signed change magnitude into event type (appearance vs. disappearance) and absolute change magnitude
State['EventType']= 'appearance'
State.set_value(State[State.ChangeMagnitude<0].index, 'EventType', 'disappearance')
State['ChangeMagnitude']= State['ChangeMagnitude'].abs()

### Loadng responses
Response= pd.read_csv(os.path.join(data_folder, 'Experiment2_FOA_Response.csv'), sep=';')

### computing response time delay and response time difference for each State Change
State['FirstResponse']= np.nan
State['LastResponse']= np.nan
State['ResponseCount']= int(0)
for iRow in range(len(State)):
    # skipping the first trial, since this the display onset rather than a change
    if np.isnan(State.ChangeMagnitude[iRow]):
        continue
        
    # identifying response time window between state changes
    response_start= State.ChangeTime[iRow]
    if iRow==len(State)-1: # last trial of the last block
        response_end= State.Duration[iRow]
    elif State.Block[iRow+1]!=State.Block[iRow]: # last trial of the block
        response_end= State.Duration[iRow]
    else: # normal trial
        response_end= State.ChangeTime[iRow+1]
        
    # identifying reponses
    iResponse= Response[(Response.ID==State.ID[iRow]) & (Response.Block==State.Block[iRow]) & (Response.Time>=response_start) & (Response.Time<response_end)].index
    State.set_value(iRow, 'ResponseCount', len(iResponse))
    if len(iResponse)>0:
        State.set_value(iRow, 'FirstResponse', Response.Time[iResponse[0]]-response_start)
        State.set_value(iRow, 'LastResponse', Response.Time[iResponse[-1]]-response_start)
    else:
        State.set_value(iRow, 'FirstResponse', np.nan)
        State.set_value(iRow, 'LastResponse', np.nan)

# computing time bracket for multikey reponses
State['TimeBracket']= State.LastResponse-State.FirstResponse        

# identifying valid trials (numbers of presses/releases matches the number of state changes)
State['Valid']= State.ChangeMagnitude==State.ResponseCount

### computing average response time brackets for two and three event
ResponseBrackets= State[(State.Valid) & (State.ChangeMagnitude>1)].groupby(['ID', 'EventType', 'ChangeMagnitude']).apply(
    lambda x: pd.Series({'MeanRT': x.TimeBracket.mean(),
                         'MinRT': x.TimeBracket.min(),
                         'MaxRT': x.TimeBracket.max(),
                         'StdRT': x.TimeBracket.std()})).reset_index()
ResponseBrackets.to_csv(os.path.join(data_folder, 'Experiment2_FOA_Estimate.csv'), sep=';', index=False)

## Importing raw results and computing subjective states and their corresponding duration

In [9]:
### loading raw data and estimate from the physically disappearing displays
results= pd.read_csv(os.path.join(data_folder, 'Experiment2_Main.csv'), sep=';')

## loading estimates for the finger-onset asynchrony (see "Experiment 1. Preliminary FOA analysis.ipynb")
FOA= ResponseBrackets

## converting event type labels to integer codes
FOA['Event']= int(0)
FOA= FOA.set_value((FOA.EventType=='appearance'), 'Event', int(1))

# marking individual events as belonging to a group of simultaneous events (or not!)
results['SimCount']= 0

# computing (in)visibility duration
subjective_state_change= pd.DataFrame(columns= ['ID', 'Block', 'BlockDuration', 'Aligned', 
                                                'MasksN',  'Trial', 'TargetCount', 'Time', 'Duration'])

for current_observer in results.ID.unique():
    for current_block in range(5, 9):
        # verbose
        print('\rObserver %s, block %d'%(current_observer, current_block), end=' ')
        
        # picking block trials
        iBlockTrials= results[(results.ID==current_observer) & (results.Block==current_block)].index
        
        ## ---------- Simultaneity ----------
        for iTrial, iRow in enumerate(iBlockTrials):
            ## checking if this is still is an unassigned event
            if results.SimCount[iRow]!= 0:
                continue
            
            ## figuring out how many consecutive events of the same type can we have
            event_time= []
            for iConsecutiveEvent in range(3):
                if (iConsecutiveEvent+iTrial>=len(iBlockTrials)) or (results.Event[iRow]!=results.Event[iRow+iConsecutiveEvent]):
                    break
                event_time.append(results.Time[iRow+iConsecutiveEvent])
            
            ## trying for three
            if len(event_time)==3:
                dT= event_time[2]-event_time[0]
                iBracket= FOA[(FOA.ID==current_observer) & (FOA.Event==results.Event[iRow]) & (FOA.ChangeMagnitude==3)].index
                if len(iBracket)>0 and dT>=FOA.MinRT[iBracket].values[0] and dT<=FOA.MaxRT[iBracket].values[0]:
                    # Match! Fill out and skip the rest of the loop
                    for iConsecutiveEvent in range(3):
                        results.set_value(iRow+iConsecutiveEvent, 'SimCount', 3)
                    continue
                    
            ## trying for twos
            if len(event_time)>=2:
                dT= event_time[1]-event_time[0]
                iBracket= FOA[(FOA.ID==current_observer) & (FOA.Event==results.Event[iRow]) & (FOA.ChangeMagnitude==2)].index
                if len(iBracket)>0 and dT>=FOA.MinRT[iBracket].values[0] and dT<=FOA.MaxRT[iBracket].values[0]:
                    # Match! Fill out and skip the rest of the loop
                    for iConsecutiveEvent in range(2):
                        results.set_value(iRow+iConsecutiveEvent, 'SimCount', 2)
                    continue
            
            ## single event, oh, well...
            results.set_value(iRow, 'SimCount', 1)
            
        ## (in)visibility duration
        # block constants
        current_entry= {'ID': current_observer, 
                        'Block': current_block, 
                        'BlockDuration': results.Duration[iBlockTrials[0]],  
                        'Aligned': results.Aligned[iBlockTrials[0]], 
                        'MasksN': results.MasksN[iBlockTrials[0]]}
        
        # adding an onset state - all visibile
        current_entry['Trial']= 0
        current_entry['TargetCount']= 3
        current_entry['Duration']= 0
        current_entry['Time']= 0
        subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)
        
        CurrentTargetCount= 3
        PrevioiusEventTime= 0
        # going through events
        for iTrial, iRow in enumerate(iBlockTrials):
            current_entry['Trial']= iTrial+1
            if results.Event[iRow]==0:
                CurrentTargetCount-= 1
            else:
                CurrentTargetCount+= 1
            current_entry['TargetCount']= CurrentTargetCount
            current_entry['Time']=  results.Time[iRow]/1000.0
            subjective_state_change.set_value(len(subjective_state_change)-1, 
                                              'Duration', 
                                              results.Time[iRow]/1000.0-PrevioiusEventTime)
            if iTrial==len(iBlockTrials)-1: # the last trial
                current_entry['Duration']= current_entry['BlockDuration']-results.Time[iRow]/1000.0
            PrevioiusEventTime= results.Time[iRow]/1000.0
            subjective_state_change.loc[subjective_state_change.shape[0]] = pd.Series(current_entry)            
            
print('\rDone!                      ')

results.to_csv(os.path.join(data_folder, 'Experiment2_Main.csv'), sep=';', index=False)
subjective_state_change.to_csv(os.path.join(data_folder, 'Experiment2_DisappearanceDuration.csv'), sep=';', index=False)

Done!                      
